In [13]:
import numpy as np
import pandas as pd

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os,csv,math,sys, joblib
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline
import collections
from joblib import Parallel, delayed
import itertools
import random
import copy
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import sklearn.model_selection, sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#import pydot
import json
import tqdm
import matplotlib
seed = 99 # To reproduce the results
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [14]:
dataset = pd.read_csv("lungcancer_10000.csv")
dataset.drop(dataset.columns[[0]], axis=1, inplace=True)
cols = dataset.columns
for i in cols:
    dataset[i] = dataset[i].map({'yes': 1, 'no': 0})

In [3]:
gt_aces = []

#ACE of asia on dysp
inp = np.array(dataset['asia']).reshape(-1,1)
out = dataset['dysp']
clf = LogisticRegression().fit(inp, out)
do_value = np.ones([len(inp), 1])
base_value = np.zeros([len(inp), 1])
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for asia on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Tub on dysp
inp = np.array(dataset['tub']).reshape(-1,1)
clf = LogisticRegression().fit(inp, out)
do_value = np.ones([len(inp), 1])
base_value = np.zeros([len(inp), 1])
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for tub on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Smoke on dysp
inp = np.array(dataset['smoke']).reshape(-1,1)
clf = LogisticRegression().fit(inp, out)
do_value = np.ones([len(inp), 1])
base_value = np.zeros([len(inp), 1])
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for smoke on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Lungcancer on dysp
inp = np.array(dataset[['lung','smoke']]).reshape(-1,2)
clf = LogisticRegression().fit(inp, out)
do_value = pd.DataFrame.copy(dataset[['lung', 'smoke']])
do_value['lung'] = 1
do_value = np.array(do_value).reshape(-1,2)
base_value = pd.DataFrame.copy(dataset[['lung', 'smoke']])
base_value['lung'] = 0
base_value = np.array(base_value).reshape(-1,2)
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for lung on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Bronc on dysp
inp = np.array(dataset[['bronc','smoke','lung', 'either']]).reshape(-1,4)
clf = LogisticRegression().fit(inp, out)
do_value = pd.DataFrame.copy(dataset[['bronc', 'smoke', 'lung', 'either']])
do_value['bronc'] = 1
do_value = np.array(do_value).reshape(-1,4)
base_value = pd.DataFrame.copy(dataset[['bronc', 'smoke', 'lung', 'either']])
base_value['bronc'] = 0
base_value = np.array(base_value).reshape(-1,4)
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for bronc on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Either on dysp
inp = np.array(dataset[['either', 'lung', 'bronc', 'smoke']]).reshape(-1,4)
clf = LogisticRegression().fit(inp, out)
do_value = pd.DataFrame.copy(dataset[['either', 'lung', 'bronc', 'smoke']])
do_value['either'] = 1
do_value = np.array(do_value).reshape(-1,4)
base_value = pd.DataFrame.copy(dataset[['either', 'lung', 'bronc', 'smoke']])
base_value['either'] = 0
base_value = np.array(base_value).reshape(-1,4)
gt_aces.append(np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))
print('ACE for either on dysp is', np.mean(clf.predict(do_value)) - np.mean(clf.predict(base_value)))


#ACE of Xray on dysp
gt_aces.append(np.mean([0.0]))
print('ACE for xray on dysp is 0.0')

ACE for asia on dysp is 0.0
ACE for tub on dysp is 1.0
ACE for smoke on dysp is 1.0
ACE for lung on dysp is 0.4911
ACE for bronc on dysp is 0.9357
ACE for either on dysp is 0.5467
ACE for xray on dysp is 0.0


In [4]:
np.save('./aces/aces_gt.npy',gt_aces,allow_pickle=True)

In [15]:
class samp_network(nn.Module):
    def __init__(self, input_size=1):
        super().__init__()
        self.input_size=input_size
        self.fc1 = nn.Linear(self.input_size, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
#order of inputs: asia	tub	smoke	lung	bronc	either	xray	dysp
class Model(nn.Module):
    def __init__(self, feature_dim, batch_size=64, device='cpu',sample_size=50000):
        super(Model, self).__init__()
        self.batchsize=batch_size

        self.causal_link_asia_tub = samp_network(input_size=1)
        self.causal_link_either_xray = samp_network(input_size=1)
        self.causal_link_smoke_bronc = samp_network(input_size=1)
        self.causal_link_smoke_lung = samp_network(input_size=1)
        self.causal_link_tub_lung_either = samp_network(input_size=2)

        self.first_layer = nn.Linear(7,4)
        self.second_layer = nn.Linear(4,2)
        self.third_layer = nn.Linear(2,2)
        self.regression_layer = nn.Linear(2, 1)
        self.sample_size = sample_size

    def forward(self, inp, phase='freeze', inde=0, alpha=0):
        if phase=='freeze':
            x = F.relu(self.first_layer(inp))
            x = F.relu(self.second_layer(x))
            x = F.relu(self.third_layer(x))
            prediction = self.regression_layer(x)
            return prediction

        elif phase=='train_dag':
            asia_sample = inp[:,0].reshape(-1,1)
            smoke_sample = inp[:,2].reshape(-1,1)
            tub_sample = self.causal_link_asia_tub(asia_sample)
            lung_sample = self.causal_link_smoke_lung(smoke_sample)
            bronc_sample = self.causal_link_smoke_bronc(smoke_sample)
            either_sample = self.causal_link_tub_lung_either(torch.cat((tub_sample, lung_sample), dim=1))
            xray_sample = self.causal_link_either_xray(either_sample)
            inp = torch.cat((asia_sample, tub_sample, smoke_sample, lung_sample, bronc_sample, either_sample, xray_sample),dim=1)
            x = F.relu(self.first_layer(inp))
            x = F.relu(self.second_layer(x))
            x = F.relu(self.third_layer(x))
            prediction = self.regression_layer(x)
            return prediction, tub_sample, lung_sample, bronc_sample, either_sample, xray_sample 

        elif phase=='sample':
            if inde == 0:
                asia_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 1:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 2:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 3:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 4:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 5:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

            elif inde == 6:
                asia_sample = inp[:,0].reshape(self.sample_size,-1)
                smoke_sample = inp[:,2].reshape(self.sample_size,-1)
                tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
                lung_sample = self.causal_link_smoke_lung(torch.tensor(smoke_sample, dtype=torch.float))
                bronc_sample = self.causal_link_smoke_bronc(torch.tensor(smoke_sample, dtype=torch.float))
                either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
                xray_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
                return inp

In [17]:
ensemble_size=5 # to get mean and std values
batch_size = 16

values = list(dataset.columns.values)
y = dataset[values[-1:]]
y = np.array(y, dtype='float32')
X = dataset[values[:-1]]
X = np.array(X, dtype='float32')

indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 2000
val_size = 1000

interval = 10
epoch = 50

X_test = X_values[-test_size:]
X_trainval = X_values[:-test_size]
X_val = X_trainval[-val_size:]
X_train = X_trainval[:-val_size]


scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

y_test = y_values[-test_size:]
y_trainval = y_values[:-test_size]
y_val = y_trainval[-val_size:]
y_train = y_trainval[:-val_size]

def binary_acc(pred, true):
    y_pred_tag = torch.round(torch.sigmoid(pred))

    correct_results_sum = (y_pred_tag == true).sum().float()
    acc = correct_results_sum/true.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [18]:
class DataSet(Dataset):
    """
    Custom dataset to load data from csv file
    """
    def __init__(self, dataframe1, dataframe2):
        self.data_points = dataframe1
        self.targets = dataframe2

    def __len__(self):
        return len(self.data_points)

    def __getitem__(self, idx):
        input_point = torch.tensor(np.array(self.data_points[idx]), dtype=torch.float)
        target_point = torch.tensor(np.array(self.targets[idx]), dtype=torch.float)
        return input_point, target_point

# ERM

In [9]:
for ensemble in range(ensemble_size):

    # Interval / Epochs
    
    loss_func = nn.BCEWithLogitsLoss()
    erm_model = Model(7,sample_size=len(dataset))
    optimizer = optim.Adam([{'params': erm_model.parameters()}], lr = 0.001, weight_decay=1e-4)

    for ep in range(0,epoch): 
        trainval = DataSet(X_train,y_train)
        train_loader = DataLoader(trainval, batch_size=batch_size)
        for input_data, target in train_loader:
            erm_model.zero_grad()
            output = erm_model(input_data)
            loss = loss_func(output,target)
            loss.backward()
            optimizer.step()
        if ep%interval == 0:
            print(ep, interval)
            val = DataSet(X_val,y_val)
            val_loader = DataLoader(val, batch_size=1)
            acc_val = 0
            acc_test = 0
            
            for input_data, target in val_loader:
                output = erm_model(input_data)
                acc = binary_acc(output, target.unsqueeze(1))
                acc_val += acc

            print ('validation accuracy:', float(acc_val/len(val_loader)))
            testval = DataSet(X_test,y_test)
            test_loader = DataLoader(testval, batch_size=1)
            
            for input_data, target in test_loader:
                output = erm_model(input_data)
                acc = binary_acc(output, target.unsqueeze(1))
                acc_test += acc
                
            print('test accuracy:', float(acc_test/len(test_loader)))
            print()
    print("************")
    torch.save(erm_model, "models/erm_lungcancer_"+str(ensemble+1))

In [9]:
for ensemble in range(ensemble_size):
    erm_model = torch.load("models/erm_lungcancer_"+str(ensemble+1))
    testval = DataSet(X_test,y_test)
    test_loader = DataLoader(testval, batch_size=1)
    acc_test = 0
    for input_data, target in test_loader:
        output = erm_model(input_data)
        acc = binary_acc(output, target.unsqueeze(1))
        acc_test += acc

    print('test accuracy:', float(acc_test/len(test_loader)))

test accuracy: 85.0999984741211
test accuracy: 85.0999984741211
test accuracy: 85.0999984741211
test accuracy: 85.0999984741211
test accuracy: 85.0999984741211


# Causal Attributions

In [220]:
n_classes=1
num_feat=7
num_alpha=2

cov = np.cov(X_values, rowvar=False)
mean_vector = np.mean(X_values, axis=0)

aces_ca_total = []

for ensemble in range(ensemble_size):
    ace_ca_total = []
    model = torch.load("models/erm_lungcancer_"+str(ensemble+1))
    for output_index in range(0,n_classes):
        for t in range(0,num_feat):
            expectation_do_x = []
            inp=copy.deepcopy(mean_vector)
            for x in np.linspace(0, 1, num_alpha):                
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
                output=model(input_torchvar)
                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_feat):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_feat)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value
            ace_ca_total.append(expectation_do_x[1] - expectation_do_x[0])
    aces_ca_total.append(ace_ca_total)
np.save('./aces/lungcancer_ca_total.npy',aces_ca_total,allow_pickle=True)

In [221]:
rmse_results = []

for ensemble in range(ensemble_size):
    rmse_results.append([rmse(gt_aces[0], aces_ca_total[ensemble][0]),
                         rmse(gt_aces[1], aces_ca_total[ensemble][1]),
                         rmse(gt_aces[2], aces_ca_total[ensemble][2]),
                         rmse(gt_aces[3], aces_ca_total[ensemble][3]),
                         rmse(gt_aces[4], aces_ca_total[ensemble][4]),
                         rmse(gt_aces[5], aces_ca_total[ensemble][5]),
                         rmse(gt_aces[6], aces_ca_total[ensemble][6])])
    
rmse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

rmse mean:  [0.07152004 0.79382408 0.99577005 0.44299555 0.73886312 0.40130374
 0.03270478]
rmse std:  [0.11000619 0.29897767 0.0166511  0.05895573 0.24109224 0.19264384
 0.04140146]
rmse all features mean:  0.49671162346976144
rmse all features std:  0.13710403401181395


# CREDO

In [56]:
prior = {0:(lambda ii:0*ii), 1:(lambda ii:1*ii), 2:(lambda ii:1*ii),
        3:(lambda ii:0.5*ii), 4:(lambda ii:1*ii), 5:(lambda ii:0.5*ii),
        6:(lambda ii:0*ii)}

def get_grad(x, prior):
    a = x.clone().detach().requires_grad_(True)
    for f in prior.keys():
        z = prior[f]
        z = torch.sum(z(a[0][f]), dim=0)
        z.backward()
    return a.grad

def get_grads_to_match(ip, prior):
    return get_grad(ip, prior)

In [87]:
for ensemble in range(ensemble_size):

    # Interval / Epochs
    
    loss_func = nn.BCEWithLogitsLoss()
    credo_model = Model(7,sample_size=len(dataset))
    optimizer = optim.Adam([{'params': credo_model.parameters()}], lr = 0.001, weight_decay=1e-4)

    for ep in range(0,epoch): 
        trainval = DataSet(X_train,y_train)
        train_loader = DataLoader(trainval, batch_size=64)
        for input_data, target in train_loader:
            credo_model.zero_grad()
            input_data.requires_grad=True
            output = credo_model(input_data)
            
            calc_grads = (autograd.grad(torch.sum(output, dim=0), input_data, retain_graph=True, create_graph=True)[0])
            grads_to_match = get_grads_to_match(input_data, prior) 
            hinge_input = torch.abs(grads_to_match - calc_grads)
            loss = loss_func(output,target) + 0.001 * torch.norm(torch.clamp(hinge_input, min=0), p=1)
            loss.backward()
            optimizer.step()
            
        if ep%interval == 0:
            print(ep, interval)
            val = DataSet(X_val,y_val)
            val_loader = DataLoader(val, batch_size=1)
            acc_val = 0
            acc_test = 0
            
            for input_data, target in val_loader:
                output = credo_model(input_data)
                acc = binary_acc(output, target.unsqueeze(1))
                acc_val += acc

            print ('validation accuracy:', float(acc_val/len(val_loader)))
            testval = DataSet(X_test,y_test)
            test_loader = DataLoader(testval, batch_size=1)
            
            for input_data, target in test_loader:
                output = credo_model(input_data)
                acc = binary_acc(output, target.unsqueeze(1))
                acc_test += acc
                
            print('test accuracy:', float(acc_test/len(test_loader)))
            print()
    print("************")
    torch.save(credo_model, "models/credo_lungcancer_"+str(ensemble+1))

0 10
validation accuracy: 40.5
test accuracy: 44.349998474121094

10 10
validation accuracy: 83.0
test accuracy: 83.94999694824219

20 10
validation accuracy: 83.0
test accuracy: 83.94999694824219

30 10
validation accuracy: 83.0
test accuracy: 83.94999694824219

40 10
validation accuracy: 83.0
test accuracy: 83.94999694824219

************


In [10]:
for ensemble in range(ensemble_size):
    credo_model = torch.load("models/credo_lungcancer_"+str(ensemble+1))
    testval = DataSet(X_test,y_test)
    test_loader = DataLoader(testval, batch_size=1)
    acc_test = 0
    for input_data, target in test_loader:
        output = credo_model(input_data)
        acc = binary_acc(output, target.unsqueeze(1))
        acc_test += acc

    print('test accuracy:', float(acc_test/len(test_loader)))

test accuracy: 83.94999694824219
test accuracy: 83.94999694824219
test accuracy: 83.94999694824219
test accuracy: 84.69999694824219
test accuracy: 83.94999694824219


In [222]:
n_classes=1
num_feat=7
num_alpha=2

cov = np.cov(X_values, rowvar=False)
mean_vector = np.mean(X_values, axis=0)

aces_credo_total = []

for ensemble in range(ensemble_size):
    ace_credo_total = []
    model = torch.load("models/credo_lungcancer_"+str(ensemble+1))
    for output_index in range(0,n_classes):
        for t in range(0,num_feat):
            expectation_do_x = []
            inp=copy.deepcopy(mean_vector)
            for x in np.linspace(0, 1, num_alpha):                
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
                output=model(input_torchvar)
                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_feat):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_feat)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value
            ace_credo_total.append(expectation_do_x[1] - expectation_do_x[0])
    aces_credo_total.append(ace_credo_total)
np.save('./aces/lungcancer_credo_total.npy',aces_credo_total,allow_pickle=True)

In [223]:
rmse_results = []

for ensemble in range(ensemble_size):
    rmse_results.append([rmse(gt_aces[0], aces_credo_total[ensemble][0]),
                         rmse(gt_aces[1], aces_credo_total[ensemble][1]),
                         rmse(gt_aces[2], aces_credo_total[ensemble][2]),
                         rmse(gt_aces[3], aces_credo_total[ensemble][3]),
                         rmse(gt_aces[4], aces_credo_total[ensemble][4]),
                         rmse(gt_aces[5], aces_credo_total[ensemble][5]),
                         rmse(gt_aces[6], aces_credo_total[ensemble][6])])

mse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

rmse mean:  [5.44726849e-04 1.00003798e+00 9.99995661e-01 4.44970928e-01
 3.78005159e-01 4.82776379e-01 4.29213047e-04]
rmse std:  [4.29021338e-04 2.08854316e-04 6.65479930e-04 9.22940608e-02
 2.40048627e-01 8.99921303e-02 3.26329729e-04]
rmse all features mean:  0.47239429233823504
rmse all features std:  0.06056635769639148


In [243]:
seed = 91 # To reproduce the results
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
interval = 5
epoch = 30
for ensemble in range(5):
    # Interval / Epochs
    trainval = DataSet(X_train,y_train)
    train_loader = DataLoader(trainval, batch_size=16)
    
    mse_loss_func = nn.MSELoss()
    loss_func = nn.BCEWithLogitsLoss()
    ahce_model = Model(7,sample_size=len(dataset))

    optimizer = optim.Adam([{'params': ahce_model.parameters()}], lr = 0.001, weight_decay=1e-4)
    losses = []
    for i in range(1):
        for ep in range(0,20):
            for phase in ['freeze', 'train_dag']: 
                for input_data, target in train_loader:
                    ahce_model.zero_grad()
                    input_data.requires_grad=False
                    if phase == 'freeze':
                        output = ahce_model(input_data)
                        loss = loss_func(output,target)
                        loss.backward(retain_graph=True)
                        optimizer.step()
                        
                    else:
                        output, tub_sample, lung_sample, bronc_sample, either_sample, xray_sample  = ahce_model(input_data, phase='train_dag')
                           
                        lam_bda = 0
                        loss = lam_bda*mse_loss_func(tub_sample, input_data[:,1])
                        loss.backward(retain_graph=True)  

                        loss = lam_bda*mse_loss_func(lung_sample, input_data[:,3])
                        loss.backward(retain_graph=True)
                        
                        loss = lam_bda*mse_loss_func(bronc_sample, input_data[:,4])
                        loss.backward(retain_graph=True)

                        loss = lam_bda*mse_loss_func(either_sample, input_data[:,5])
                        loss.backward(retain_graph=True)
                        
                        loss = lam_bda*torch.sqrt(mse_loss_func(xray_sample, input_data[:,6]))
                        loss.backward(retain_graph=True)                   
    
                        loss = loss_func(output,target)
                        loss.backward(retain_graph=True)
                        optimizer.step()

                if ep%interval == 0:
                    print(ep, interval)
                    val = DataSet(X_val,y_val)
                    val_loader = DataLoader(val, batch_size=1)
                    acc_val = 0
                    acc_test = 0

                    for input_data, target in val_loader:
                        output = ahce_model(input_data)
                        acc = binary_acc(output, target.unsqueeze(1))
                        acc_val += acc

                    print ('validation accuracy:', float(acc_val/len(val_loader)))
                    testval = DataSet(X_test,y_test)
                    test_loader = DataLoader(testval, batch_size=1)

                    for input_data, target in test_loader:
                        output = ahce_model(input_data)
                        acc = binary_acc(output, target.unsqueeze(1))
                        acc_test += acc

                    print('test accuracy:', float(acc_test/len(test_loader)))
                    print()

    torch.save(ahce_model, "./models/ahce_lungcancer_"+str(ensemble+1))
#     torch.save(ahce_model, "./models/ahce_lungcancer_1")

0 5
validation accuracy: 57.5
test accuracy: 55.54999923706055

0 5
validation accuracy: 63.20000076293945
test accuracy: 61.650001525878906

5 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

5 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

10 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

10 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

15 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

15 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

0 5
validation accuracy: 57.5
test accuracy: 55.54999923706055

0 5
validation accuracy: 57.5
test accuracy: 55.54999923706055

5 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

5 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

10 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

10 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

15 5
validation accuracy: 85.5
test accuracy: 85.0999984741211

15 5
validation accuracy: 85.5

In [244]:
n_classes=1
num_c=7#no. of features
num_alpha=2

aces_ahce_total = []
for ensemble in range(5):
    print("ensemble: ", ensemble)
    ace_ahce_total = []
    model =  torch.load("./models/ahce_lungcancer_"+str(ensemble+1))
    for output_index in range(0,n_classes):#For every class
        #plt.figure()
        for t in range(0,num_c):#For every feature
            print("feature: ", t)
            expectation_do_x = []
            for x in np.linspace(0, 1, num_alpha):
                X_values[:,t] = x
                sample_data = model(X_values, phase='sample', inde=t, alpha=x).detach().numpy()
                cov = np.cov(sample_data, rowvar=False)
                means = np.mean(sample_data, axis=0)
                cov=np.array(cov)
                mean_vector = np.array(means)
                inp=copy.deepcopy(mean_vector)
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)

                output=model(input_torchvar)

                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_c):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_c)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value

            ace_ahce_total.append(expectation_do_x[1] - expectation_do_x[0])

    aces_ahce_total.append(ace_ahce_total)
np.save('./aces/lungcancer_ahce_total.npy',aces_ahce_total,allow_pickle=True)

ensemble:  0
feature:  0
feature:  1
feature:  2
feature:  3
feature:  4


/tmp/ipykernel_5118/265272099.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tub_sample = self.causal_link_asia_tub(torch.tensor(asia_sample, dtype=torch.float))
/tmp/ipykernel_5118/265272099.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  either_sample = self.causal_link_tub_lung_either(torch.cat((torch.tensor(tub_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float)), dim=1))
/tmp/ipykernel_5118/265272099.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xray_sample = self.causal_link_either_xray(torch.t

feature:  5
feature:  6
ensemble:  1
feature:  0
feature:  1
feature:  2


/tmp/ipykernel_5118/265272099.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xray_sample = self.causal_link_either_xray(torch.tensor(either_sample, dtype=torch.float))
/tmp/ipykernel_5118/265272099.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.cat((torch.tensor(asia_sample, dtype=torch.float), torch.tensor(tub_sample, dtype=torch.float), torch.tensor(smoke_sample, dtype=torch.float), torch.tensor(lung_sample, dtype=torch.float), torch.tensor(bronc_sample, dtype=torch.float), torch.tensor(either_sample, dtype=torch.float), torch.tensor(xray_sample, dtype=torch.float)),dim=1)
/tmp/ipykernel_5118/265272099.py:115: UserWarning: To copy construct from a tensor, it i

feature:  3
feature:  4
feature:  5
feature:  6
ensemble:  2
feature:  0
feature:  1
feature:  2
feature:  3
feature:  4
feature:  5
feature:  6
ensemble:  3
feature:  0
feature:  1
feature:  2
feature:  3
feature:  4
feature:  5
feature:  6
ensemble:  4
feature:  0
feature:  1
feature:  2
feature:  3
feature:  4
feature:  5
feature:  6


In [240]:
rmse_results = []

for ensemble in [0,1,2,3]:
    rmse_results.append([rmse(gt_aces[0], aces_ahce_total[ensemble][0]),
                         rmse(gt_aces[1], aces_ahce_total[ensemble][1]),
                         rmse(gt_aces[2], aces_ahce_total[ensemble][2]),
                         rmse(gt_aces[3], aces_ahce_total[ensemble][3]),
                         rmse(gt_aces[4], aces_ahce_total[ensemble][4]),
                         rmse(gt_aces[5], aces_ahce_total[ensemble][5]),
                         rmse(gt_aces[6], aces_ahce_total[ensemble][6])])

mse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

rmse mean:  [0.05796246 0.5865253  0.56491578 0.77649596 1.1145698  0.65574369
 0.08098546]
rmse std:  [0.06275573 0.29022294 0.33369286 0.75975439 0.51841772 0.2375601
 0.12799542]
rmse all features mean:  0.5481712078000819
rmse all features std:  0.33291416532225815


In [246]:
rmse_results = []

for ensemble in [0,1,2,3]:
    rmse_results.append([rmse(gt_aces[0], aces_ahce_total[ensemble][0]),
                         rmse(gt_aces[1], aces_ahce_total[ensemble][1]),
                         rmse(gt_aces[2], aces_ahce_total[ensemble][2]),
                         rmse(gt_aces[3], aces_ahce_total[ensemble][3]),
                         rmse(gt_aces[4], aces_ahce_total[ensemble][4]),
                         rmse(gt_aces[5], aces_ahce_total[ensemble][5]),
                         rmse(gt_aces[6], aces_ahce_total[ensemble][6])])

mse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

rmse mean:  [0.05796246 0.5865253  0.56491578 0.77649596 1.1145698  0.65574369
 0.08098546]
rmse std:  [0.06275573 0.29022294 0.33369286 0.75975439 0.51841772 0.2375601
 0.12799542]
rmse all features mean:  0.5481712078000819
rmse all features std:  0.33291416532225815
